# Feature Engineering – Retail Forecasting (Gross Sales)

Mục tiêu: Sinh đặc trưng theo SKU × tuần (W-MON) an toàn leakage, dựa trên gợi ý ở notebook EDA Gross.

Nội dung chính:
- Chuẩn bị dữ liệu weekly (reindex fill-0 theo từng SKU)
- Snapshot thống kê TRAIN (median, p99 theo SKU) để tránh leakage
- Tạo bộ feature MVP: lags, rolling, EMA, calendar, price/promo
- Lưu bộ dữ liệu engineered ra `data/processed/`


In [1]:
# Setup & Config
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Paths
ROOT = Path('..')
DATA_PROCESSED = ROOT / 'data' / 'processed'
INPUT_CSV = DATA_PROCESSED / 'processed_data_gross.csv'
OUTPUT_FE_CSV = DATA_PROCESSED / 'fe_weekly_gross.csv'
SNAPSHOT_DIR = DATA_PROCESSED / 'snapshots'
SNAPSHOT_DIR.mkdir(parents=True, exist_ok=True)

# Time config
WEEK_FREQ = 'W-MON'
TRAIN_END = pd.Timestamp('2011-06-01')
VAL_END = pd.Timestamp('2011-09-01')  # kept for reference

print('Input:', INPUT_CSV)
print('Output:', OUTPUT_FE_CSV)
print('Train end:', TRAIN_END.date())


Input: ..\data\processed\processed_data_gross.csv
Output: ..\data\processed\fe_weekly_gross.csv
Train end: 2011-06-01


In [2]:
# Load data

df = pd.read_csv(INPUT_CSV)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
print('Loaded!', df.shape)
print('Date range:', df['InvoiceDate'].min(), '->', df['InvoiceDate'].max())

# Use sales-only rows (Quantity > 0)
df = df[df['Quantity'] > 0].copy()

# Basic columns check
required_cols = {'Invoice','StockCode','InvoiceDate','Quantity','Price','Country'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f'Missing required columns: {missing}')


Loaded! (1002894, 14)
Date range: 2009-12-01 07:45:00 -> 2011-12-09 12:50:00


In [3]:
# Train/Val/Test split markers (for leakage-safe snapshots)

train_mask = df['InvoiceDate'] < TRAIN_END
val_mask = (df['InvoiceDate'] >= TRAIN_END) & (df['InvoiceDate'] < VAL_END)
test_mask = df['InvoiceDate'] >= VAL_END

print('Split sizes:', train_mask.sum(), val_mask.sum(), test_mask.sum())


Split sizes: 681742 107752 213400


In [4]:
# Helper: build weekly panel per SKU with continuous index and fill zeros

def build_weekly_panel(raw_df: pd.DataFrame, freq: str = WEEK_FREQ) -> pd.DataFrame:
    g = (raw_df
         .groupby([pd.Grouper(key='InvoiceDate', freq=freq), 'StockCode'])
         .agg(Quantity=('Quantity','sum'),
              Price=('Price','mean'),
              Country=('Country', lambda x: x.mode().iat[0] if len(x.mode()) else x.iloc[0]))
         .reset_index())
    # Reindex per SKU to continuous weekly range
    panels = []
    for sku, gd in g.groupby('StockCode'):
        gd = gd.sort_values('InvoiceDate')
        full_index = pd.date_range(gd['InvoiceDate'].min(), gd['InvoiceDate'].max(), freq=freq)
        gd = gd.set_index('InvoiceDate').reindex(full_index)
        gd.index.name = 'InvoiceDate'
        gd['StockCode'] = sku
        # Fill zeros for Quantity, forward-fill Country, keep Price as is (will use median snapshot for indices)
        gd['Quantity'] = gd['Quantity'].fillna(0.0)
        gd['Country'] = gd['Country'].ffill().bfill()
        panels.append(gd.reset_index())
    out = pd.concat(panels, ignore_index=True)
    return out[['InvoiceDate','StockCode','Country','Quantity','Price']]

weekly = build_weekly_panel(df)
print('Weekly shape:', weekly.shape)
weekly.head(3)


Weekly shape: (328788, 5)


,InvoiceDate,StockCode,Country,Quantity,Price
0,2009-12-07,10002,United Kingdom,142.0,0.850000
1,2009-12-14,10002,United Kingdom,59.0,1.133333
2,2009-12-21,10002,United Kingdom,12.0,0.850000


In [ ]:
# Leakage-safe snapshots: compute SKU medians and p99 on TRAIN only
train_weeks = weekly[weekly['InvoiceDate'] < TRAIN_END].copy()

median_price_sku = (train_weeks.groupby('StockCode')['Price']
                    .median().rename('median_price_sku'))

# p99 per SKU for Quantity (on train) – for winsorizing flags
p99_qty_sku = (train_weeks.groupby('StockCode')['Quantity']
               .quantile(0.99).rename('p99_qty_sku'))

snapshot = pd.concat([median_price_sku, p99_qty_sku], axis=1).reset_index()
SNAPSHOT_PATH = SNAPSHOT_DIR / 'snap_gross_train_stats.csv'
snapshot.to_csv(SNAPSHOT_PATH, index=False)
print('Saved snapshot:', SNAPSHOT_PATH)

Saved snapshot: ..\data\processed\snapshots\snap_gross_train_stats.csv


In [7]:
# Feature builder functions (MVP set) – leakage-safe (use t-1 info for windows)

from typing import Tuple

def add_price_promo_features(dfw: pd.DataFrame, snapshot_df: pd.DataFrame) -> pd.DataFrame:
    out = dfw.merge(snapshot_df, on='StockCode', how='left')
    out['median_price_sku'] = out['median_price_sku'].replace(0, np.nan)
    # Use previous week's price for price-based features
    out['price_prev'] = out.groupby('StockCode')['Price'].shift(1)
    out['price_index'] = out['price_prev'] / out['median_price_sku']
    out['log_price'] = np.log(out['price_prev'].clip(lower=1e-6))
    out['promo_flag'] = (out['price_index'] < 0.8).astype('int8')
    out['promo_depth'] = np.clip(1 - out['price_index'], 0, None)
    # Price change signal (previous vs previous-1) to avoid using current-week info
    out['price_prev_prev'] = out.groupby('StockCode')['Price'].shift(2)
    out['d_price_pct_1w'] = (out['price_prev'] - out['price_prev_prev']) / out['price_prev_prev']
    # Rolling stats computed on previous prices only
    out['roll_mean_price_4w'] = out.groupby('StockCode')['price_prev'].transform(lambda x: x.rolling(4, min_periods=1).mean())
    out['roll_std_price_8w'] = out.groupby('StockCode')['price_prev'].transform(lambda x: x.rolling(8, min_periods=2).std())
    out['price_zscore_8w'] = (out['price_prev'] - out['roll_mean_price_4w']) / (out['roll_std_price_8w'].replace(0, np.nan))
    return out

def add_lag_rolling_ema(dfw: pd.DataFrame) -> pd.DataFrame:
    g = dfw.sort_values(['StockCode','InvoiceDate']).copy()
    for k in [1,2,3,4,5,8,12]:
        g[f'qty_lag_{k}'] = g.groupby('StockCode')['Quantity'].shift(k)
    # Compute rolling/EMA on previous quantities (t-1)
    g['qty_prev'] = g.groupby('StockCode')['Quantity'].shift(1)
    g['roll_mean_qty_8w'] = g.groupby('StockCode')['qty_prev'].transform(lambda x: x.rolling(8, min_periods=1).mean())
    g['roll_mean_qty_12w'] = g.groupby('StockCode')['qty_prev'].transform(lambda x: x.rolling(12, min_periods=1).mean())
    g['roll_std_qty_8w'] = g.groupby('StockCode')['qty_prev'].transform(lambda x: x.rolling(8, min_periods=2).std())
    g['ema_qty_4w'] = g.groupby('StockCode')['qty_prev'].transform(lambda x: x.ewm(span=4, adjust=False).mean())
    g['ema_qty_8w'] = g.groupby('StockCode')['qty_prev'].transform(lambda x: x.ewm(span=8, adjust=False).mean())
    return g

def add_intermittency_features(dfw: pd.DataFrame) -> pd.DataFrame:
    g = dfw.sort_values(['StockCode','InvoiceDate']).copy()
    # Weeks since last sale computed from previous weeks only
    def weeks_since_last_from_prev(arr: np.ndarray) -> np.ndarray:
        last = -1
        out = np.empty_like(arr, dtype='int32')
        for i, v in enumerate(arr):
            if v > 0:
                last = i
            out[i] = (i - last) if last >= 0 else i+1
        return out
    prev_has_sale = g.groupby('StockCode')['Quantity'].shift(1).fillna(0) > 0
    g['weeks_since_last_sale'] = prev_has_sale.groupby(g['StockCode']).transform(lambda s: weeks_since_last_from_prev(s.astype('int8').values))
    # Rolling zero rates on previous quantities
    prev_qty = g.groupby('StockCode')['Quantity'].shift(1).fillna(0)
    g['zero_rate_8w'] = prev_qty.groupby(g['StockCode']).transform(lambda x: x.rolling(8, min_periods=1).apply(lambda w: np.mean(w==0)))
    g['zero_rate_12w'] = prev_qty.groupby(g['StockCode']).transform(lambda x: x.rolling(12, min_periods=1).apply(lambda w: np.mean(w==0)))
    return g

def add_calendar_features(dfw: pd.DataFrame) -> pd.DataFrame:
    g = dfw.copy()
    woy = g['InvoiceDate'].dt.isocalendar().week.astype(int)
    g['woy_sin'] = np.sin(2*np.pi*woy/52)
    g['woy_cos'] = np.cos(2*np.pi*woy/52)
    g['quarter'] = g['InvoiceDate'].dt.quarter.astype('int16')
    g['is_q4'] = (g['quarter']==4).astype('int8')
    return g

def add_quality_flags(dfw: pd.DataFrame) -> pd.DataFrame:
    g = dfw.copy()
    # Extreme based on last week's qty vs train p99 (avoid using current y)
    g['extreme_qty_flag'] = (g.get('qty_lag_1', pd.Series(index=g.index, dtype=float)) > g['p99_qty_sku']).astype('int8')
    # Price jump based on previous price stats
    g['price_jump_flag'] = ((g['price_zscore_8w'] > 3) | (np.abs(g['price_index']-1) > 0.5)).astype('int8')
    return g


In [8]:
# Build features (MVP) – ensure no leakage by relying on TRAIN snapshots only

fe = weekly.copy()
fe = add_price_promo_features(fe, snapshot)
fe = add_lag_rolling_ema(fe)
fe = add_intermittency_features(fe)
fe = add_calendar_features(fe)
fe = add_quality_flags(fe)

# Target: Quantity (gross, >=0). For models using log1p target, compute separately in training code.
fe['y'] = fe['Quantity'].astype('float32')

# Minimal dtype downcast
int_cols = [c for c in fe.columns if fe[c].dtype == 'int64']
for c in int_cols:
    fe[c] = fe[c].astype('int32')

print('FE shape:', fe.shape)
fe.head(3)


FE shape: (328788, 40)


,InvoiceDate,StockCode,Country,Quantity,Price,median_price_sku,p99_qty_sku,price_prev,price_index,log_price,...,weeks_since_last_sale,zero_rate_8w,zero_rate_12w,woy_sin,woy_cos,quarter,is_q4,extreme_qty_flag,price_jump_flag,y
0,2009-12-07,10002,United Kingdom,142.0,0.850000,0.85,845.89,NaN,NaN,NaN,...,1,1.000000,1.000000,-2.393157e-01,0.970942,4,1,0,0,142.0
1,2009-12-14,10002,United Kingdom,59.0,1.133333,0.85,845.89,0.850000,1.000000,-0.162519,...,0,0.500000,0.500000,-1.205367e-01,0.992709,4,1,0,0,59.0
2,2009-12-21,10002,United Kingdom,12.0,0.850000,0.85,845.89,1.133333,1.333333,0.125163,...,0,0.333333,0.333333,6.432491e-16,1.000000,4,1,0,0,12.0


In [9]:
# Save engineered dataset

cols_order = ['InvoiceDate','StockCode','Country','y','Quantity','Price','median_price_sku','price_index','log_price',
              'promo_flag','promo_depth','d_price_pct_1w','roll_mean_price_4w','roll_std_price_8w','price_zscore_8w',
              'qty_lag_1','qty_lag_2','qty_lag_3','qty_lag_4','qty_lag_5','qty_lag_8','qty_lag_12',
              'roll_mean_qty_8w','roll_mean_qty_12w','roll_std_qty_8w','ema_qty_4w','ema_qty_8w',
              'weeks_since_last_sale','zero_rate_8w','zero_rate_12w','woy_sin','woy_cos','quarter','is_q4',
              'p99_qty_sku','extreme_qty_flag','price_jump_flag']

# Keep only existing columns
final_cols = [c for c in cols_order if c in fe.columns]
fe_out = fe[final_cols].copy()
fe_out.to_csv(OUTPUT_FE_CSV, index=False)
print('Saved:', OUTPUT_FE_CSV, 'with shape', fe_out.shape)


Saved: ..\data\processed\fe_weekly_gross.csv with shape (328788, 37)


### Ghi chú
- Tất cả thống kê dùng trong feature (`median_price_sku`, `p99_qty_sku`) được ước lượng trên TRAIN rồi áp cho toàn bộ.
- Các feature có cửa sổ (lag/rolling/EMA) đều dựa trên dữ liệu quá khứ (đến t−1).
- Mở rộng tiếp theo: country-aware price index, returns_ratio, co-occurrence/bundle, trend 8w.
